In [ ]:
## Script used to check Pauli Twirling Pass from qiskit.

In [1]:
from qiskit.dagcircuit import DAGCircuit
from qiskit.circuit import QuantumCircuit, QuantumRegister, Gate
from qiskit.circuit.library import CXGate, ECRGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.quantum_info import Operator, pauli_basis
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, coherent_unitary_error)

import numpy as np
 
from typing import Iterable, Optional

In [2]:
class PauliTwirl(TransformationPass):
    """Add Pauli twirls to two-qubit gates."""
 
    def __init__(
        self,
        gates_to_twirl: Optional[Iterable[Gate]] = None,
    ):
        """
        Args:
            gates_to_twirl: Names of gates to twirl. The default behavior is to twirl all
                two-qubit basis gates, `cx` and `ecr` for IBM backends.
        """
        if gates_to_twirl is None:
            gates_to_twirl = [CXGate(), ECRGate()]
        self.gates_to_twirl = gates_to_twirl
        self.build_twirl_set()
        super().__init__()
 
    def build_twirl_set(self):
        """
        Build a set of Paulis to twirl for each gate and store internally as .twirl_set.
        """
        self.twirl_set = {}
 
        # iterate through gates to be twirled
        for twirl_gate in self.gates_to_twirl:
            twirl_list = []
 
            # iterate through Paulis on left of gate to twirl
            for pauli_left in pauli_basis(2):
                # iterate through Paulis on right of gate to twirl
                for pauli_right in pauli_basis(2):
                    # save pairs that produce identical operation as gate to twirl
                    if (Operator(pauli_left) @ Operator(twirl_gate)).equiv(
                        Operator(twirl_gate) @ pauli_right
                    ):
                        twirl_list.append((pauli_left, pauli_right))
 
            self.twirl_set[twirl_gate.name] = twirl_list
 
    def run(
        self,
        dag: DAGCircuit,
    ) -> DAGCircuit:
        # collect all nodes in DAG and proceed if it is to be twirled
        twirling_gate_classes = tuple(
            gate.base_class for gate in self.gates_to_twirl
        )
        for node in dag.op_nodes():
            if not isinstance(node.op, twirling_gate_classes):
                continue
 
            # random integer to select Pauli twirl pair
            pauli_index = np.random.randint(
                0, len(self.twirl_set[node.op.name])
            )
            twirl_pair = self.twirl_set[node.op.name][pauli_index]
 
            # instantiate mini_dag and attach quantum register
            mini_dag = DAGCircuit()
            register = QuantumRegister(2)
            mini_dag.add_qreg(register)
 
            # apply left Pauli, gate to twirl, and right Pauli to empty mini-DAG
            mini_dag.apply_operation_back(
                twirl_pair[0].to_instruction(), [register[0], register[1]]
            )
            mini_dag.apply_operation_back(node.op, [register[0], register[1]])
            mini_dag.apply_operation_back(
                twirl_pair[1].to_instruction(), [register[0], register[1]]
            )
 
            # substitute gate to twirl node with twirling mini-DAG
            dag.substitute_node_with_dag(node, mini_dag)
 
        return dag

In [3]:
def circular_ansatz_mirrored(N, reps=1, fix_2q=False): 
    qc = QuantumCircuit(N)
    for _ in range(reps):
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N-1, -1, -1):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(np.pi/2, i)
    for i in range(N):
        qc.rz(np.pi/2, i)
    return qc

num_qubits = 10
qc = circular_ansatz_mirrored(num_qubits)

## Ideal

In [4]:
np.random.seed(0)
paulis = ["".join(np.random.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)) for _ in range(num_qubits)]
np.random.seed(0)
coeffs = np.random.random(len(paulis))
observable = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

In [5]:
#Aer Backend
backend = AerSimulator(method='statevector',device = "GPU", seed_simulator = 0)
ideal_estimator = Estimator(mode=backend)

In [6]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,seed_transpiler=0)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)
job = ideal_estimator.run([(isa_circuit, isa_observable)])


In [7]:
# Get results for the first (and only) PUB
ideal_res = job.result()[0]
print(f">>> Expectation value: {ideal_res.data.evs}")

>>> Expectation value: -0.02461625332245609


## Noisy

In [8]:
noise_model = NoiseModel()

# # Depolarizing Noise
# cx_depolarizing_prob = 0.1
# noise_model.add_all_qubit_quantum_error(
#     depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
# )

#Coherent Noise
epsilon = 0.1

err_cx = QuantumCircuit(2)
err_cx.cx(0,1)
err_cx.p(epsilon, 0)
err_cx.p(epsilon, 1)

err_cx.cx(0,1)
err_cx.p(-epsilon, 0)
err_cx.p(-epsilon, 1)

err_cx = Operator(err_cx)
noise_model.add_all_qubit_quantum_error(
    coherent_unitary_error(err_cx), ["cx"]
)

noisy_backend = AerSimulator(method='statevector', device='GPU',seed_simulator = 0,
                       noise_model=noise_model)

noisy_estimator = Estimator(mode=noisy_backend)
# (noisy_estimator.options.simulator.noise_model) = noise_model #NOTE: Remove this?

In [9]:
pm = generate_preset_pass_manager(backend=noisy_backend, optimization_level=1)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)

job = noisy_estimator.run([(isa_circuit, isa_observable)])
noisy_res = job.result()[0]

print(f">>> Expectation value: {noisy_res.data.evs}")

>>> Expectation value: -0.02450730416948204


In [10]:
abs_error_diff = abs(noisy_res.data.evs - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff}")

Absolute error difference: 0.00010894915297405064


## Pauli Twirling

In [11]:
pm = PassManager([PauliTwirl()])
twirled_qcs = [pm.run(qc) for _ in range(5)]
isa_observable= observable.apply_layout(twirled_qcs[0].layout)

In [12]:
estimator = Estimator(noisy_backend)
isa_circ_obs_list = [(qc,isa_observable) for qc in twirled_qcs]

In [13]:
job = estimator.run(isa_circ_obs_list)
job_result = job.result()
exp_vals = []
for idx in range(len(job_result)):
    pub_result = job_result[idx]
    exp_vals.append(pub_result.data.evs)

exp_vals=np.array(exp_vals)
np.average(exp_vals)

0.004303395106181466

In [14]:
abs_error_diff_pt = abs(np.average(exp_vals) - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff_pt}")

Absolute error difference: 0.028919648428637555
